In [ ]:
import numpy as np
import pandas as pd
import json
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
#from collections import Counter

In [ ]:
data = []
for line in open('arxiv-metadata-oai-snapshot.json', 'r'):
    data.append(json.loads(line))
df = pd.DataFrame.from_records(data)

In [ ]:
df

,id,submitter,authors,title,comments,journal-ref,doi,abstract,report-no,categories,versions
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,A fully differential calculation in perturba...,ANL-HEP-PR-07-12,[hep-ph],"[v1, v2]"
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,"We describe a new algorithm, the $(k,\ell)$-...",None,[math.CO cs.CG],"[v1, v2]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,The evolution of Earth-Moon system is descri...,None,[physics.gen-ph],"[v1, v2, v3]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,We show that a determinant of Stirling cycle...,None,[math.CO],[v1]
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,In this paper we show how to compute the $\L...,None,[math.CA math.FA],[v1]
...,...,...,...,...,...,...,...,...,...,...,...
3360979,supr-con/9608008,Ruslan Prozorov,"R. Prozorov, M. Konczykowski, B. Schmidt, Y. Y...",On the origin of the irreversibility line in t...,"19 pages, LaTex, 6 PostScript figures; Author'...",None,10.1103/PhysRevB.54.15530,We report on measurements of the angular dep...,None,[supr-con cond-mat.supr-con],[v1]
3360980,supr-con/9609001,Durga P. Choudhury,"Durga P. Choudhury, Balam A. Willemsen, John S...",Nonlinear Response of HTSC Thin Film Microwave...,"4 pages, LaTeX type, Uses IEEE style files, 60...",None,10.1109/77.620744,The non-linear microwave surface impedance o...,None,[supr-con cond-mat.supr-con],[v1]
3360981,supr-con/9609002,Durga P. Choudhury,"Balam A. Willemsen, J. S. Derov and S.Sridhar ...",Critical State Flux Penetration and Linear Mic...,"20 pages, LaTeX type, Uses REVTeX style files,...",None,10.1103/PhysRevB.56.11989,The vortex contribution to the dc field (H) ...,None,[supr-con cond-mat.supr-con],[v1]
3360982,supr-con/9609003,Hasegawa Yasumasa,Yasumasa Hasegawa (Himeji Institute of Technol...,Density of States and NMR Relaxation Rate in A...,"7 pages, 4 PostScript Figures, LaTeX, to appea...",None,10.1143/JPSJ.65.3131,We show that the density of states in an ani...,None,[supr-con cond-mat.supr-con],[v1]


In [ ]:
df['abstract'] = df['abstract'].str.replace('\n', ' ',regex=False)
df['abstract'] = df['abstract'].str.replace('-', '',regex=False)
df['abstract'] = df['abstract'].str.replace('_', '',regex=False)
df['abstract'] = df['abstract'].str.replace('(', '',regex=False)
df['abstract'] = df['abstract'].str.replace(')', '',regex=False)
df['abstract'] = df['abstract'].str.replace('@', '',regex=False)
df['abstract'] = df['abstract'].str.replace('#', '',regex=False)
df['abstract'] = df['abstract'].str.replace('$', '',regex=False)
df['abstract'] = df['abstract'].str.replace('%', '',regex=False)
df['abstract'] = df['abstract'].str.replace('^', '',regex=False)
df['abstract'] = df['abstract'].str.replace('&', '',regex=False)
df['abstract'] = df['abstract'].str.replace('*', '',regex=False)
df['abstract'] = df['abstract'].str.replace('+', '',regex=False)
df['abstract'] = df['abstract'].str.replace('=', '',regex=False)
df['abstract'] = df['abstract'].str.replace('?', '',regex=False)
df['abstract'] = df['abstract'].str.replace('|', '',regex=False)
df['abstract'] = df['abstract'].str.replace('\\', '',regex=False)
df['abstract'] = df['abstract'].str.replace('/', '',regex=False)
df['abstract'] = df['abstract'].str.replace(':', '',regex=False)
df['abstract'] = df['abstract'].str.replace(':', '',regex=False)
df['abstract'] = df['abstract'].str.replace('<', '',regex=False)
df['abstract'] = df['abstract'].str.replace('>', '',regex=False)
df['abstract'] = df['abstract'].str.replace('[', '',regex=False)
df['abstract'] = df['abstract'].str.replace(']', '',regex=False)
df['abstract'] = df['abstract'].str.replace('{', '',regex=False)
df['abstract'] = df['abstract'].str.replace('}', '',regex=False)

df["abstract"][0]

In [ ]:
#df["abstract"] = df["title"]+"."+df["abstract"]

In [ ]:
df.isnull().sum()

In [ ]:
df["categories"].unique()

In [ ]:
df_cs = df

In [ ]:
drop_idx = []
for index, row in df_cs.iterrows():
    if 'cs.CL' not in row['categories']:
        drop_idx.append(index)
df_cs = df_cs.drop(drop_idx).reset_index(drop=True)
df_cs.to_csv('arxiv_cs-CL.csv', index=False)
df_cs

In [ ]:
df_ph = df

In [ ]:
drop_idx = []
for index, row in df_ph.iterrows():
    if 'ph' not in row['categories']:
        drop_idx.append(index)
df_ph = df_ph.drop(drop_idx).reset_index(drop=True)
df_ph.to_csv('arxiv_ph.csv', index=False)
df_ph

In [ ]:
df["abstract"][1]

In [ ]:
df_math = df

In [ ]:
drop_idx = []
for index, row in df_math.iterrows():
    if 'math' not in row['categories']:
        drop_idx.append(index)
df_math = df_math.drop(drop_idx).reset_index(drop=True)
df_math.to_csv('arxiv_math.csv', index=False)
df_math

In [ ]:
abstract_cs = df_cs['abstract']
abstract_df_cs = abstract_cs.to_frame(name='abstract')
abstract_cs

In [ ]:
abstract_ph = df_ph['abstract']
abstract_df_ph = abstract_ph.to_frame(name='abstract')
abstract_ph

In [ ]:
abstract_math = df_math['abstract']
abstract_df_math = abstract_math.to_frame(name='abstract')
abstract_math

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as stop_word

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stop_word)
tfidf_matrix_cs = vectorizer.fit_transform(abstract_cs)
print(tfidf_matrix_cs.shape)

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stop_word)
tfidf_matrix_ph = vectorizer.fit_transform(abstract_ph)
print(tfidf_matrix_ph.shape)

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stop_word)
tfidf_matrix_math = vectorizer.fit_transform(abstract_math)
print(tfidf_matrix_math.shape)

In [ ]:
cosine_sim_linear_kernel_cs = linear_kernel(tfidf_matrix_cs, tfidf_matrix_cs)

In [ ]:
pd.DataFrame(tfidf_matrix.toarray())

,0
0,1.0


In [ ]:
#cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
#print(cosine_sim)

In [ ]:
cosine_sim_linear_kernel = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
print(cosine_sim_linear_kernel)

[[1.]]


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer()
tokenizer.fit_on_texts(abstract)
tokenized_documents=tokenizer.texts_to_sequences(abstract)
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=100,padding='post')
vocab_size=len(tokenizer.word_index)+1
print (tokenized_paded_documents[0])

In [ ]:
abstract

In [ ]:
documents = ['Machine learning is the study of computer algorithms that improve automatically through experience.\
Machine learning algorithms build a mathematical model based on sample data, known as training data.\
The discipline of machine learning employs various approaches to teach computers to accomplish tasks \
where no fully satisfactory algorithm is available.',
'Machine learning is closely related to computational statistics, which focuses on making predictions using computers.\
The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning.',
'Machine learning involves computers discovering how they can perform tasks without being explicitly programmed to do so. \
It involves computers learning from data provided so that they carry out certain tasks.',
'Machine learning approaches are traditionally divided into three broad categories, depending on the nature of the "signal"\
or "feedback" available to the learning system: Supervised, Unsupervised and Reinforcement',
'Software engineering is the systematic application of engineering approaches to the development of software.\
Software engineering is a computing discipline.',
'A software engineer creates programs based on logic for the computer to execute. A software engineer has to be more concerned\
about the correctness of the program in all the cases. Meanwhile, a data scientist is comfortable with uncertainty and variability.\
Developing a machine learning application is more iterative and explorative process than software engineering.'
]

documents_df=pd.DataFrame(documents,columns=['documents'])

In [ ]:
stop_words_l=stopwords.words('english')
abstract['abstract']=abstract.abstract.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

tfidfvectoriser=TfidfVectorizer()
tfidfvectoriser.fit(abstract.abstract)
tfidf_vectors=tfidfvectoriser.transform(abstract.abstract)

pairwise_similarities=np.dot(tfidf_vectors,tfidf_vectors.T).toarray()
pairwise_differences=euclidean_distances(tfidf_vectors)

def most_similar(doc_id,similarity_matrix,matrix):
    print (f'abstract: {abstract.iloc[doc_id]["abstract"]}')
    print ('\n')
    print ('Similar abstract:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        print('\n')
        print (f'abstract: {abstract.iloc[ix]["abstract"]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

most_similar(0,pairwise_similarities,'Cosine Similarity')
most_similar(0,pairwise_differences,'Euclidean Distance')


In [ ]:
print (tfidf_vectors[0].toarray())
print (pairwise_similarities.shape)
print (pairwise_similarities[0][:])

In [ ]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

document_embeddings = sbert_model.encode(abstract['abstract'])

pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

most_similar(0,pairwise_similarities,'Cosine Similarity')
most_similar(0,pairwise_differences,'Euclidean Distance')